In [1]:
import pandas as pd
import numpy as np
import os
import swifter
from common.preprocessing.dataframe import *
from common.bio.amino_acid import *

In [2]:
STRUCTURE_COL = "sst8"
ROOT = "..\\..\\data\\protein\\structure\\secondary_structure\\"
#DATA_SOURCES = ROOT + "data_sources\\2018-06-06-pdb-intersect-pisces.csv"
DATA_SOURCES = ROOT + "data_sources\\cullpdb+profile_6133.npy"
SAVE_PATH = ROOT

In [3]:
data = np.load(DATA_SOURCES)
test = np.load(ROOT + "data_sources\\cb513+profile_split1.npy")

In [5]:
def unique(a):
    order = np.lexsort(a.T)
    a = a[order]
    diff = np.diff(a, axis=0)
    ui = np.ones(len(a), 'bool')
    ui[1:] = (diff != 0).any(axis=1) 
    return a[ui]

In [6]:
data = unique(data)

In [7]:
data.shape, test.shape

((5926, 39900), (514, 39900))

In [15]:
data[0:5926].shape

(5926, 39900)

In [18]:
data = np.append(data,test, axis=0)

In [20]:
data.shape

(6440, 39900)

In [21]:
data = data.reshape(6440,700,57)
data.shape

(6440, 700, 57)

In [22]:
labels = np.argmax(data[:,:,22:31], axis =2)

In [23]:
sequences = np.argmax(data[:, :, np.r_[0:22]], axis =2)

In [24]:
data[:, :, np.r_[0:22]][0][0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [25]:
sequences[0][0]

4

In [26]:
sequences = sequences+100
labels = labels + 100

In [27]:
acid_mapping = {100:1, 101:2, 102:4, 103:3, 104:6, 105:5, 106:8, 107:7, 108:9, 109:11, 110:10, 111:12, 
 112:14, 113:13, 114:16, 115:15, 116:17, 117:19, 118:18, 119:20, 120:21, 121:0}


In [28]:
for k in acid_mapping.keys():
    sequences[sequences == k] = acid_mapping[k]

In [29]:
sequences[0][0], np.argmax(data[:, :, np.r_[0:22]], axis =2)[0][0]

(6, 4)

In [30]:
structure_mapping = {100:2, 101:1, 102:3, 103:4, 104:6, 105:5, 106:7, 107:8, 108:0}

In [31]:
{0: '0', 1: 'B', 2: 'C', 3: 'E', 4: 'G', 5: 'H', 6: 'I', 7: 'S', 8: 'T'}

{0: '0', 1: 'B', 2: 'C', 3: 'E', 4: 'G', 5: 'H', 6: 'I', 7: 'S', 8: 'T'}

In [32]:
for k in structure_mapping.keys():
    labels[labels == k] = structure_mapping[k]

In [33]:
labels[0][0], np.argmax(data[:,:,22:31], axis =2)[0][0]

(2, 0)

In [34]:
#filtered_data = filter_non_standard_amino_acids(filtered_data, "seq")

In [35]:
mutations = data[:,:,35:55]
pssm = data[:,:,35:55]

In [36]:
mutations[0][0]

array([0.92689884, 0.05215356, 0.07446795, 0.06537533, 0.01448573,
       0.99451375, 0.04393982, 0.01506799, 0.08548915, 0.01448573,
       0.03389517, 0.15577585, 0.0541667 , 0.0772722 , 0.04269665,
       0.91606164, 0.1665886 , 0.03661483, 0.01695654, 0.26894143])

In [37]:
mutations = np.power(mutations, 2.5)

In [38]:
mutations = mutations/np.sum(mutations, axis=2)[:,:,None]

c:\users\donatas\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [39]:
mutations = mutations.cumsum(axis=2)

In [40]:
data[3,33,35:55]

array([1.44240505e-03, 1.66558093e-04, 1.22939877e-03, 6.56119315e-03,
       3.03446926e-04, 5.47281292e-04, 5.01646893e-03, 2.56019615e-04,
       2.19257250e-01, 6.17015059e-04, 4.19357279e-03, 4.07013763e-03,
       6.95627765e-04, 6.15383744e-01, 9.99830008e-01, 2.92947027e-03,
       1.94606138e-03, 4.34882444e-04, 3.59657075e-04, 2.68941432e-01])

In [41]:
(data[3,33,35:55]/7.260719653218985).cumsum(axis=0)

array([1.98658689e-04, 2.21598301e-04, 3.90920190e-04, 1.29457623e-03,
       1.33636918e-03, 1.41174481e-03, 2.10265000e-03, 2.13791092e-03,
       3.23356407e-02, 3.24206206e-02, 3.29981905e-02, 3.35587599e-02,
       3.36545669e-02, 1.18409767e-01, 2.56113749e-01, 2.56517217e-01,
       2.56785243e-01, 2.56845138e-01, 2.56894673e-01, 2.93935275e-01])

In [42]:
mutations[3][33]

array([5.82426790e-08, 5.85065779e-08, 9.75686425e-08, 2.66784794e-06,
       2.66903024e-06, 2.67419500e-06, 3.98796269e-06, 3.98873574e-06,
       1.65963490e-02, 1.65963560e-02, 1.65971954e-02, 1.65979744e-02,
       1.65979838e-02, 2.35569845e-01, 9.72351286e-01, 9.72351629e-01,
       9.72351752e-01, 9.72351755e-01, 9.72351757e-01, 1.00000000e+00])

In [44]:
lengths = (sequences==0).argmax(axis=1)
lengths = lengths.reshape(6440,1)

In [45]:
sequences.shape, labels.shape, lengths.shape, mutations.shape

((6440, 700), (6440, 700), (6440, 1), (6440, 700, 20))

In [46]:
np.random.choice(np.arange(1,20+1), p = data[3,153,35:55]/sum(data[3,153,35:55]))

5

## Save to numpy

In [47]:
train_idx = 5926
test_idx = 6440

In [48]:
train = np.stack([sequences[0:train_idx], labels[0:train_idx]], axis= 1)
# val = np.stack([sequences[test_idx:val_idx], labels[test_idx:val_idx]], axis= 1)
test = np.stack([sequences[train_idx:test_idx], labels[train_idx:test_idx]], axis= 1)

In [49]:
train.shape

(5926, 2, 700)

In [51]:
os.makedirs(os.path.join(SAVE_PATH, "cullpdb" ,"train"), exist_ok=True)

In [52]:
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"train", "data.npy"), train)
# np.save(os.path.join(SAVE_PATH, "cullpdb" ,"val", "data.npy"), val)
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"test", "data.npy"), test)

In [53]:
np.load(os.path.join(SAVE_PATH, "cullpdb" ,"train", "data.npy"))[1].shape

(2, 700)

## Lengths

In [54]:
train = lengths[0:train_idx]
test = lengths[train_idx:test_idx]

In [67]:
lengths[train_idx:test_idx][lengths[train_idx:test_idx] > 512]

array([526, 534, 544, 576], dtype=int64)

In [55]:
test.shape

(514, 1)

In [56]:
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"train", "lengths.npy"), train)
# np.save(os.path.join(SAVE_PATH, "cullpdb" ,"val", "lengths.npy"), val)
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"test", "lengths.npy"), test)

## Mutations

In [57]:
train = mutations[0:train_idx]
test = mutations[train_idx:test_idx]

In [58]:
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"train", "mutations_cumsum.npy"), train)
# np.save(os.path.join(SAVE_PATH, "cullpdb" ,"val", "mutations_cumsum.npy"), val)
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"test", "mutations_cumsum.npy"), test)

In [60]:
np.load(os.path.join(SAVE_PATH, "cullpdb" ,"train", "mutations_cumsum.npy"))[1].shape

(700, 20)

## PSSM

In [61]:
train = pssm[0:train_idx]
test = pssm[train_idx:test_idx]

In [62]:
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"train", "pssm.npy"), train)
# np.save(os.path.join(SAVE_PATH, "cullpdb" ,"val", "pssm.npy"), val)
np.save(os.path.join(SAVE_PATH, "cullpdb" ,"test", "pssm.npy"), test)